----
----
---

# BURADA YAPILACAK OLAN AÇIKLAMALARIN NEREDEYSE TAMAMINA YAKINI UYGULAMA-5 İÇİN YAPILACAKTIR.EĞER DİREKT BU .İPYNB DOSYASINDAN BAŞLADIYSANIZ UYGULAMA 1'e DÖNMENİZİ ÖNERİRİM.ZATEN İÇERİSİNDE GEREKLİ AÇIKLAMALARI YİNE .ipynb FORMATINDAKİ DOSYADAN BULABİLİRSİNİZ
---
----
----


# LibTorch tarafında gördüklerimiz

torch::Tensor

torch::rand(...)

torch::jit::script::Module

torch::jit::load(...)

module.to(torch::kCPU)

module.eval()

tensor.sizes()

torch::softmax(...)

tensor.argmax(...)

tensor.item<int>()

c10::Error (exception tipi)

# CMake tarafında gördüklerimiz

cmake_minimum_required(...)

project(...)

set(CMAKE_PREFIX_PATH "...")

find_package(Torch REQUIRED)

find_package(OpenCV REQUIRED)

add_executable(...)

target_link_libraries(... "${TORCH_LIBRARIES}" ${OpenCV_LIBS})

set_property(TARGET ... PROPERTY CXX_STANDARD 17)

add_custom_command(... copy_directory ...) ile DLL kopyalama

OpenCV_DIR ayarlama

# OpenCV tarafında gördüklerimiz

cv::Mat

cv::imread(...)

cv::imshow(...)

cv::waitKey(...)

cv::resize(...)

cv::cvtColor(...)

convertTo(..., CV_32F, 1.0/255.0)

# Preprocess & Tensor dönüşümü tarafında gördüklerimiz

torch::from_blob(...) ile cv::Mat → torch::Tensor

NHWC → NCHW dönüşümü için tensor.permute(...)

tensor.contiguous()

tensor.clone()

# Inference çağrısı tarafında gördüklerimiz

std::vector<torch::jit::IValue>

inputs.push_back(input_tensor)

module.forward(inputs).toTensor()

# Genel C++ iskeletinde gördüklerimiz

std::cout ile log yazma ([INFO] Program basladi., vb.)

std::ifstream ile dosya var mı kontrolü (model_ts.pt, test.jpg)

try { ... } catch (const c10::Error& ...)

try { ... } catch (const std::exception& ...)

std::cin.get() ile program sonu bekleme

----------

# 1) torch::Tensor – Bunu neden kullandım?

* Bu projede görüntüyü modele verebilmem için veriyi PyTorch’un anlayacağı forma sokmam gerekiyordu. PyTorch tarafında tüm veriler tensor üzerinden işleniyor. Bu yüzden:

**Ben bu projede tensor kullandım çünkü modelimin inputu tensör formatında olmalıydı.**

### Daha net şekilde:

* Benim yüklediğim görüntü cv::Mat formatındaydı.

* Modelim ise tensör bekliyor.

* Bu ikisi farklı yapılar.

* O yüzden resmi tensöre dönüştürmeden modele veremezdim.

### Ayrıca tensör sayesinde:

* float32’e çevirme,

* normalize etme,

* boyut düzenleme,

* NCHW dönüşümü,

* GPU/CPU taşıma

gibi işlemleri tek bir yapı üzerinden yapabildim.

### Kısacası:

* “Ben tensörü veriyi PyTorch modelimin anlayacağı forma sokmak için kullandım.”

> ## Aşağıdaki koddan çıkarım alarak bu ifadeleri daha net inceleyelim:

```cpp
torch::Tensor preprocess_image(const cv::Mat &img_bgr, int target_size)
{
    cv::Mat img;

    img_bgr.copyTo(img);

    cv::resize(img, img, cv::Size(target_size, target_size));

    cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

    cv::Mat img_float;
    img.convertTo(img_float, CV_32F, 1.0f / 255.0f);

    auto tensor_image = torch::from_blob(
        img_float.data,
        {1, img_float.rows, img_float.cols, 3},
        torch::TensorOptions().dtype(torch::kFloat32));

    tensor_image = tensor_image.permute({0, 3, 1, 2});
    tensor_image = tensor_image.contiguous().clone();

    return tensor_image;
}


----

## 2️⃣ preprocess_image içinde tensör: torch::from_blob ve permute
```cpp
torch::Tensor preprocess_image(const cv::Mat &img_bgr, int target_size)
```


Burada ben, fonksiyonun dönüş tipini torch::Tensor yaptım; çünkü:

* Bu fonksiyonu çağırdığımda elimde direkt modele verebileceğim bir input tensörü olsun istiyorum.

### a) torch::from_blob(...)
```cpp
auto tensor_image = torch::from_blob(
    img_float.data,
    {1, img_float.rows, img_float.cols, 3}, // [N, H, W, C]
    torch::TensorOptions().dtype(torch::kFloat32));
```


Burada ben şunu yapıyorum:

* OpenCV’de cv::Mat içinde duran img_float.data ham piksel verisini alıyorum,

* Ona bir şekil veriyorum: [1, H, W, 3] (NHWC),

* Ve bunu tensöre sarıyorum.

**Yani burada ben, OpenCV’nin ham bellek bölgesini PyTorch tensörüne bağlıyorum. Henüz kopya almıyorum, direkt aynı belleği kullanıyorum.**



### b) tensor_image.permute({0, 3, 1, 2})
```cpp
tensor_image = tensor_image.permute({0, 3, 1, 2}); // [N, C, H, W]
```


Burada ben, tensörün eksenlerini değiştiriyorum:

* OpenCV’den gelirken tensörüm [N, H, W, C] formatındaydı.

* PyTorch modeli ise [N, C, H, W] formatını bekliyor.

- O yüzden ben **tensörün boyut sırasını değiştiriyorum ki modelle uyumlu olsun.**


### c) contiguous().clone()
```cpp
tensor_image = tensor_image.contiguous().clone();
```


Burada ise ben şunu yapıyorum:

* from_blob ile oluşturduğum tensör hâlâ OpenCV’nin belleğine bağlı.

* Ben, bu bağlantıyı kesmek istiyorum. Çünkü fonksiyon bitince cv::Mat yok olursa tensör çöp belleğe bakmasın.

Bu yüzden:

* contiguous() ile belleği düzgün, ardışık bir düzene sokuyorum,

* clone() ile tamamen kendi kopyasına sahip, bağımsız bir tensör oluşturuyorum.

### Sonuç:
**Ben bu fonksiyondan, tamamen bağımsız, modeli beslemeye hazır bir torch::Tensor döndürüyorum.**

----

## 3️⃣ main içinde tensör: input_tensor
```cpp
torch::Tensor input_tensor = preprocess_image(img_bgr, target_size);
```


Burada ben:

* OpenCV’den okuduğum img_bgr’yi alıyorum,

* preprocess_image ile işleyip,

**Sonuçta elimde [1, 3, 64, 64] şeklinde bir tensör tutuyorum.**


### Hemen ardından boyut kontrolü yapıyorum:
```cpp
auto sizes = input_tensor.sizes();
if (!(sizes.size() == 4 &&
      sizes[0] == 1 &&
      sizes[1] == 3 &&
      sizes[2] == target_size &&
      sizes[3] == target_size))
{
    // hata...
}
```


Burada da:

* Ben tensörün boyutunun, modelin beklediği formatta olup olmadığını doğruluyorum.

* Yanlışsa programı bilinçli şekilde durduruyorum, “model niye saçma çıktı veriyor?” diye uğraşmıyorum.

----
## 4️⃣ run_inference içinde tensör: input ve output
```cpp
torch::Tensor run_inference(torch::jit::script::Module &module,
                            const torch::Tensor &input)
{
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input);

    torch::Tensor output = module.forward(inputs).toTensor();
    return output;
}
```


Burada ben şunu yapıyorum:

* Parametre olarak input tensörü alıyorum, çünkü model doğrudan tensör bekliyor.

* Ama TorchScript forward fonksiyonu  
>std::vector<torch::jit::IValue> istediği için,

* Ben tensörü IValue içine koyup inputs vektörüne push ediyorum.

* module.forward(inputs) diyorum, bu bana yine bir IValue döndürüyor.

* Ben bunu toTensor() ile tekrar çıktı tensörüne çeviriyorum.

Yani bu fonksiyonda:

**Ben tensörü modele veriyorum, modelin ürettiği çıktıyı yine tensör olarak geri alıyorum.**

----

## 5️⃣ Postprocess: softmax, argmax, item<int>()

```cpp
torch::Tensor probs = torch::softmax(output, 1);
torch::Tensor pred_class = probs.argmax(1);

std::cout << "Predicted class index: "
          << pred_class.item<int>() << "\n";
```


Burada:

* output zaten tensör (logit değerleri).

* Ben softmax ile bu tensörü olasılık dağılımına çeviriyorum → probs.

* Sonra argmax(1) ile, satır bazında en yüksek olasılığa sahip sınıfın indeksini buluyorum → pred_class.

* pred_class hâlâ tensör (şekli [1]).

* item<int>() ile, ben bu tek elemanlı tensörü normal C++ int değerine çeviriyorum.

Yani burada tensörü:

* Önce matematiksel işlem için kullanıyorum (softmax, argmax),

* Sonra sonucu ekrana yazdırmak için primitive tipe indiriyorum.

----
# Bu kodda ben tensörü:

* OpenCV’den gelen resmi PyTorch’un anlayacağı forma sokmak,

* Bu resmi TorchScript modeline input olarak vermek,

* Modelden gelen çıktıları üzerinde matematiksel işlem yapmak (softmax, argmax),

- Ve son olarak bu sonucu C++ tarafında normal bir tamsayıya çevirip kullanmak için kullandım.

----
-----

##  tensor.contiguous() – Ben bu fonksiyonu neden çağırdım?

Kodda:
```cpp
tensor_image = tensor_image.permute({0, 3, 1, 2}); // [N, C, H, W]
tensor_image = tensor_image.contiguous().clone();
```


Burada contiguous() kısmı şunu yapıyor:

* permute() çağırınca tensorun görünür boyut sırası değişiyor, ama çoğu zaman PyTorch sadece strides’ları değiştirip aynı belleği kullanıyor.

* Yani permute sonrası tensor çoğu durumda non-contiguous oluyor.

- Ben de, bellek düzenini düzeltmek için tensor_image = tensor_image.contiguous(); diyorum.

### Bu ne anlama geliyor?

* Eğer tensor zaten contiguous ise → yeni kopya almıyor, aynı belleği döndürüyor.

* Eğer non-contiguous ise → arkada yeni bir buffer oluşturuyor, veriyi düzgün sırada buraya kopyalıyor.

Benim bakış açım:

**“Ben contiguous() çağırarak, tensörün bellekte tertemiz, ardışık bir düzende olmasını garanti altına alıyorum. Böylece hem performans hem de uyumluluk açısından başımı ağrıtacak bir şey bırakmıyorum.”**

---

## clone() – Ben niye üstüne bir de clone() koydum?

Aynı satırda:
```cpp
tensor_image = tensor_image.contiguous().clone();
```


clone() şunu yapıyor:

* Tensorun tamamen yeni bir kopyasını oluşturuyor.

Yani:

* Yeni bir data adresi

* İçerik birebir aynı

* Artık başka bir kaynağa bağlı değil

Bizim durumda kritik nokta şu:

**torch::from_blob(img_float.data, ...) ile ben aslında OpenCV’nin RAM’ine referans veren bir tensor oluşturdum.**

* Bu tensorun belleğinin sahibi OpenCV.

>Eğer cv::Mat scope dışına çıkarsa veya free olursa → tensör çöp adresi gösteriyor.

Bu yüzden:

>**“Ben clone() ile, tensörün verisini tamamen yeni bir bellek alanına kopyalıyorum. Böylece OpenCV ölse bile, tensörüm kendi başına hayatta kalıyor.”**

Yani:

* from_blob → dış kaynağa bağlı tensor (borrowed memory)

* clone → kendi RAM’ine sahip tensor (owned memory)

Benim kafamdaki özet:

>**“contiguous → bellek düzenini düzelt, clone → bu belleğin sahibi ben olayım.”**

### Eğer bunu birine tek cümleyle anlatacak olsam şöyle derdim:

**“Bu projede from_blob ile OpenCV’nin RAM’ini tensöre sardım, permute ile boyut sırasını modele uydurdum, contiguous().clone() ile hem belleği düzgün hale getirip hem de verinin gerçek sahibi olarak kendim yeni bir kopyasını aldım; gerisi zaten softmax–argmax–item zinciriyle çıktı okumak.”**


## ✔ En sade hali:

**“contiguous = düzelt**

**clone = sahiplen”**

-----

# 1️⃣ from_blob – Parametreleriyle ben burada ne yapıyorum?

KOD:
```cpp
auto tensor_image = torch::from_blob(
    img_float.data,
    {1, img_float.rows, img_float.cols, 3}, // [N, H, W, C]
    torch::TensorOptions().dtype(torch::kFloat32));

tensor_image = tensor_image.permute({0, 3, 1, 2}); // [N, C, H, W]
```


## a) img_float.data

Burada ben şunu yapıyorum:

* img_float zaten cv::Mat ve içinde görüntünün piksel verisi var.

* img_float.data ile ben bu verinin RAM adresini veriyorum.

* Yani diyorum ki: “Bu adresi tensor verisi olarak kullan.”

#### Eğer bunu yapmazsam:

* Zaten from_blob çalışmaz; tensörün nereden veri okuyacağını bilemez.

* Başka bir buffer versem, yanlış resimden veri okur.

#### Daha önemli risk:
**Bu pointer cv::Mat’e ait. Eğer ben daha sonra clone() yapmazsam ve img_float scope dışına çıkarsa → tensör çöp pointer’a bakmaya başlar, runtime’da patlar (bozuk sonuç, crash, undefined behavior).**


## b) {1, img_float.rows, img_float.cols, 3}

>Bu ikinci parametre tamamen şekil (shape).

Ben burada tensörün boyutlarını şuna göre veriyorum:

* 1 → batch size (N) → ben tek resim verdiğim için 1

* img_float.rows → H (yükseklik)

* img_float.cols → W (genişlik)

* 3 → C (kanal sayısı – RGB)

**Yani ben bu tensörü [N, H, W, C] = [1, H, W, 3] olarak tanımlıyorum.**

### Bu shape parametresiyle ben aslında şunu yapıyorum:

**“Ben RAM’de duran düz piksel verisine, ‘sen 1 tane H×W çözünürlüklü, 3 kanallı bir görüntüsün’ diye anlam giydiriyorum.”**

## c)torch::TensorOptions().dtype(torch::kFloat32)

Burada ben şunu söylüyorum:

* “Bu tensörün eleman tipi float32 olacak.”

### Niye böyle?

* Ben biraz önce img.convertTo(img_float, CV_32F, 1.0f / 255.0f); ile resmi zaten float32 + normalize 0–1 aralığına getirdim.

Dolayısıyla tensörün de float32 olması gerekiyor ki:

* Bellekteki veri ile tensörün tipi uyumlu olsun.

* Modelim de float bekliyor, int ile uğraşmıyor.

### Bunu yapmazsam:

#### Mesela dtype(torch::kUInt8) dersem:

* Bellekteki veri aslında float32 iken, tensör onu uint8 olarak yorumlar.

* Aynı byte’ları farklı tipte okumuş olurum → tüm piksel değerleri saçma hale gelir.

* Modelin gördüğü input tamamen bozulur → çıktı çöp.

#### Hiç dtype vermezsem:

* Bazı durumlarda PyTorch tip tahmin etmeye çalışır, ama burada kritik olan ben zaten elimdeki verinin tipini biliyorum, o yüzden net belirtmem lazım.

---
# 2️⃣ permute({0, 3, 1, 2}) – Burada ben ne yapıyorum?

from_blob sonrası tensörüm:

* Şekil: [1, H, W, 3] → NHWC

Modelim ise PyTorch standardına göre:

* [1, 3, H, W] → NCHW

Yani ben permute({0,3,1,2}) ile şunu diyorum:

* eksen → yerinde kalsın (N)

* eksen → ikinci sıraya gelsin (C)

* eksen → üçüncü sıraya gelsin (H)

* eksen → dördüncü sıraya gelsin (W)

**Sonuç: [N, C, H, W] = [1, 3, H, W]**

Yani:

>“Ben permute ile tensörün eksenlerini, OpenCV formatından (H,W,C) PyTorch’un beklediği formata (C,H,W) çeviriyorum.”

### Eğer permute’i yanlış parametreyle yaparsam?

Örneğin:
```cpp
tensor_image.permute({0, 1, 2, 3});   // hiçbir şey değiştirmiyor
tensor_image.permute({0, 2, 3, 1});   // [N, W, C, H] gibi saçma bir şey
```


* Şekil yine 4 boyutlu olur ama eksen dizilimi modele uymaz.

* conv katmanları beklenen kanalı yanlış eksenden okuyabilir.

* Output ya çöp olur ya da boyut uyuşmazlığından patlar.

---
# 3️⃣ Bu iki satırın “net” özeti 

Ben bu iki satırda aslında şunu yapıyorum:

### from_blob ile:

* OpenCV’nin float32 normalize görüntü verisini,

* [1, H, W, 3] şeklinde, NHWC formatında bir tensöre sarıyorum.

* Data pointer olarak img_float.data kullanıyorum.

### permute({0,3,1,2}) ile:

* Bu tensörü [1, 3, H, W] formatına çeviriyorum.

* Yani kanalı üçüncü boyuttan ikinci boyuta taşıyorum.

Eğer:

**shape’i yanlış verirsem → tensörün fiziksel belleği ile mantıksal boyutu uymaz, model ya patlar ya da saçma çalışır.**

**dtype’ı yanlış verirsem → piksel değerlerini yanlış tipte okurum, model çöp input görür.**

**permute yapmazsam → model NCHW beklerken NHWC verir, tensör boyutları uyuşmaz veya anlamlı olmayan sonuç çıkar.**